In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

In [ ]:
slice_name = 'MySlice'
[site1,site2]  = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# Node1
node1 = slice.add_node(name=node1_name, site=site1, disk=100, image='default_ubuntu_22')
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

# Node2
node2 = slice.add_node(name=node2_name, site=site2, disk=100, image='default_ubuntu_22')
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)

#Submit Slice Request
slice.submit()

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node1_addr = node1.get_interface(network_name=network_name).get_ip_addr()
node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()


print("node1_addr = " + str(node1_addr))
print("node2_addr = " + str(node2_addr))

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

In [ ]:
commands = [
    'sudo apt update -y',
    'sudo apt-get install -y python3-pip',
]

print("Installing pip3")
for command in commands:
    print(f"++++ {command}")
    stdout, stderr = node1.execute(command)
    stdout, stderr = node2.execute(command)

In [ ]:
# SSH to two nodes

# Install gRPC
# $ python3 -m pip install grpcio

# Install gRPC tools
# $ python -m pip install grpcio-tools

# Clone the repository to get the example code:
# $ git clone -b v1.61.0 --depth 1 --shallow-submodules https://github.com/grpc/grpc

# Navigate to the "hello, world" Python example:
# $ cd grpc/examples/python/helloworld

# We consider node 2 as client, modify node's greeter_client.py like below:

# """The Python implementation of the GRPC helloworld.Greeter client."""

# from __future__ import print_function

# import logging

# import grpc
# import helloworld_pb2
# import helloworld_pb2_grpc


# def run():
#     # NOTE(gRPC Python Team): .close() is possible on a channel and should be
#     # used in circumstances in which the with statement does not fit the needs
#     # of the code.
#     print("Will try to greet world ...")
#     with grpc.insecure_channel("192.168.1.2:50051") as channel:
#         stub = helloworld_pb2_grpc.GreeterStub(channel)
#         response = stub.SayHello(helloworld_pb2.HelloRequest(name="you"))
#     print("Greeter client received: " + response.message)


# if __name__ == "__main__":
#     logging.basicConfig()
#     run()

# We consider node 1 as server, modify node's greeter_server.py like below: 

# """The Python implementation of the GRPC helloworld.Greeter server."""

# from concurrent import futures
# import logging

# import grpc
# import helloworld_pb2
# import helloworld_pb2_grpc


# class Greeter(helloworld_pb2_grpc.GreeterServicer):
#     def SayHello(self, request, context):
#         return helloworld_pb2.HelloReply(message="Hello, %s!" % request.name)


# def serve():
#     port = "50051"
#     server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
#     helloworld_pb2_grpc.add_GreeterServicer_to_server(Greeter(), server)
#     server.add_insecure_port("192.168.1.2:" + port)
#     server.start()
#     print("Server started, listening on " + port)
#     server.wait_for_termination()


# if __name__ == "__main__":
#     logging.basicConfig()
#     serve()


In [ ]:
# Delele Slice after the experiment is done
slice.delete()